# This notebook contains our code for the German Credit Dataset (Sections 4.3 and 4.5 of the paper)

### The dataset is available from the UCI repository at [this url](https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

from sklearn import linear_model
import xgboost

from sklearn.model_selection import train_test_split

import dice_ml
from dice_ml.utils import helpers  # helper functions

import facct_util as futil

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load and prepare the data

In [3]:
feature_names = ['checking account status', 'Duration', 'Credit history', 'Purpose', 'Credit amount',
           'Savings account/bonds', 'Present employment since', 'Installment rate in percentage of disposable income',
           'Personal status and sex', 'Other debtors / guarantors', 'Present residence since',
           'Property', 'Age in years', 'Other installment plans', 'Housing', 'Number of existing credits at this bank',
           'Job', ' Number of people being liable to provide maintenance for', 'Telephone', 'foreign worker']

cagegorical_features = ['checking account status', 'Credit history', 'Purpose', 'Credit amount',
           'Savings account/bonds', 'Present employment since', 
           'Personal status and sex', 'Other debtors / guarantors',
           'Property', 'Other installment plans', 'Housing', 
           'Job', 'Telephone', 'foreign worker']

cts_features = ['Duration', 'Credit amount', 'Installment rate in percentage of disposable income', 'Age in years',
                'Number of existing credits at this bank', ' Number of people being liable to provide maintenance for']

columns = [*feature_names, 'target' ]

In [4]:
len(feature_names), len(cagegorical_features) + len(cts_features)

(20, 20)

In [5]:
data = pd.read_csv("german.data", sep=' ', header=None)
data.columns = columns
y = data['target']-1
X = data
X = X.drop('target', axis=1)
cat_columns = X.select_dtypes(['object']).columns
X[cat_columns] = X[cat_columns].apply(lambda x: x.astype('category').cat.codes)
X.head()

,checking account status,Duration,Credit history,Purpose,Credit amount,Savings account/bonds,Present employment since,Installment rate in percentage of disposable income,Personal status and sex,Other debtors / guarantors,Present residence since,Property,Age in years,Other installment plans,Housing,Number of existing credits at this bank,Job,Number of people being liable to provide maintenance for,Telephone,foreign worker
0,0,6,4,4,1169,4,4,4,2,0,4,0,67,2,1,2,2,1,1,0
1,1,48,2,4,5951,0,2,2,1,0,2,0,22,2,1,1,2,1,0,0
2,3,12,4,7,2096,0,3,2,2,0,3,0,49,2,1,1,1,2,0,0
3,0,42,2,3,7882,0,3,2,2,2,4,1,45,2,2,1,2,2,0,0
4,0,24,3,0,4870,0,2,3,2,0,4,3,53,2,2,2,2,2,0,0


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Train a gradient boosted tree

In [8]:
gbtree = xgboost.XGBClassifier(n_estimators=300, max_depth=5, use_label_encoder=False, random_state=0)
gbtree.fit(X_train, Y_train)
sklearn.metrics.accuracy_score(Y_test, gbtree.predict(X_test))

[13:56:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.76

### Train logistic regresion

In [9]:
log_regr = linear_model.LogisticRegression(penalty='none', max_iter=10000)
log_regr.fit(X_train, Y_train)

sklearn.metrics.accuracy_score(Y_test, log_regr.predict(X_test))

0.745

### Prepare the data for the DiCE framework

In [10]:
df_train = X_train.copy()
df_train.insert(20, 'outcome', Y_train.values)
df_test = X_test.copy()

In [11]:
d = dice_ml.Data(dataframe=df_train, continuous_features=cts_features, outcome_name='outcome', enable_categorical=True)

In [12]:
df_train['outcome'] = df_train['outcome'].astype('bool')
for cf in cts_features:
    df_train[cf] = df_train[cf].astype('float32')

In [13]:
X_train_dice = d.normalize_data(d.one_hot_encode_data(X_train))
X_test_dice = d.normalize_data(d.one_hot_encode_data(X_test))

In [14]:
X_test_dice.head()

,Duration,Credit amount,Installment rate in percentage of disposable income,Age in years,Number of existing credits at this bank,Number of people being liable to provide maintenance for,checking account status_0,checking account status_1,checking account status_2,checking account status_3,...,Housing_1,Housing_2,Job_0,Job_1,Job_2,Job_3,Telephone_0,Telephone_1,foreign worker_0,foreign worker_1
993,0.470588,0.204083,1.000000,0.196429,0.0,0.0,1,0,0,0,...,1,0,0,0,0,1,0,1,1,0
859,0.073529,0.183064,0.000000,0.125000,0.0,1.0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,1
298,0.205882,0.124629,0.666667,0.428571,0.0,0.0,0,0,0,1,...,1,0,0,0,1,0,0,1,1,0
553,0.117647,0.096016,1.000000,0.142857,0.0,0.0,0,1,0,0,...,1,0,0,0,1,0,1,0,1,0
672,0.823529,0.556619,0.333333,0.410714,0.0,0.0,0,0,0,1,...,1,0,0,0,0,1,0,1,1,0


### How many counterfactual explanations that work for the gradient boosted tree also work for logistic regression?

In [15]:
m_gb = dice_ml.Model(gbtree, backend="sklearn", model_type='classifier')
exp_random_gb = dice_ml.Dice(d, m_gb, method="random")

In [16]:
num_diverse_explanations = 20
N = 50
cross_counts = 0
for i in range(N):
    query_instance= df_test[i:i+1]
    exp = exp_random_gb.generate_counterfactuals(query_instance, total_CFs=num_diverse_explanations, desired_class="opposite")
    for j in range(num_diverse_explanations):
        df = exp._cf_examples_list[0].final_cfs_df_sparse[j:j+1] # the j-th counterfactual explanation for the gbtree
        x = df.values[0, 0:-1].reshape(1, 20)
        if log_regr.predict(x)[0] == gbtree.predict(x)[0]:
            cross_counts += 1
            
print(f'Fraction of counterfactual explanations that also work for logistic regression: {cross_counts/N/num_diverse_explanations}')

100%|██████████| 1/1 [00:00<00:00,  3.84it/s]

Fraction of counterfactual explanations that also work for logistic regression: 0.41


### How many different counterfactual explanations exist for a typical individual that was rejected the credit?
#### We say that two counterfactual explanations are 'different' if they modify different subsets of features. This notion could of course be refined.

In [19]:
df_test_gb_denied = df_test.drop(df_test.index) # remove data but perserve structure in terms of column names and data types

for i in range(200):
    query_instance= df_test[i:i+1]
    if gbtree.predict(query_instance.values.reshape(1, 20))[0] == 0:
        df_test_gb_denied = df_test_gb_denied.append(query_instance)

In [34]:
for i_obs in range(20):
    query_instance = df_test_gb_denied[i_obs:i_obs+1]
    exp = exp_random_gb.generate_counterfactuals(query_instance, total_CFs=1000, desired_class="opposite", verbose=False)
    cfs = []
    for i_cf in range(exp._cf_examples_list[0].final_cfs_df.shape[0]):
        cf = exp._cf_examples_list[0].final_cfs_df[i_cf:i_cf+1]
        cf_features = []
        for feature in feature_names:
            if cf[feature].values[0] != query_instance[feature].values[0]:
                cf_features.append(feature)
        cfs.append(cf_features)
    print(f'Number of different counterfactual explanations for individual {i_obs}: ', len(np.unique(np.array(cfs, dtype=object))))

  0%|          | 0/1 [00:00<?, ?it/s]

Number of different counterfactual explanations for individual 0:  861


  0%|          | 0/1 [00:00<?, ?it/s]

Number of different counterfactual explanations for individual 1:  966


  0%|          | 0/1 [00:00<?, ?it/s]

...

Number of different counterfactual explanations for individual 17:  973


  0%|          | 0/1 [00:00<?, ?it/s]

Number of different counterfactual explanations for individual 18:  977


100%|██████████| 1/1 [00:09<00:00,  9.46s/it]


Number of different counterfactual explanations for individual 19:  901
